# Azure Postgres Flex with vector search
This sample shows how to use azure postgres (Flex) native vector search capabilities for RAG applications.  
### Prerequisite: install python libraries
- Please make sure all the libraries found in requirements.txt are installed in your python environment. 
- Rename example.env to llm.env and enter your credentials in llm_flex.env
- Whitelist your IP to access your PostGres dv. Add you IP in "Networking" section of your PostGRes resource on the [Azure Portal](https://ms.portal.azure.com/)
- To apply vector search, please install vector extensions for your db. You may follow this [link](https://learn.microsoft.com/en-us/azure/postgresql/flexible-server/concepts-extensions) to add vector to the allow-list of your db extensions.  

# Load environment variables and keys 

In [ ]:
from dotenv import dotenv_values

# specify the name of the .env file name 
env_name = "llm_pgvector.env" # change to your own .env file name
config = dotenv_values(env_name)

# Connect to Flex Postgres (PG) 

In [ ]:
import psycopg2
from psycopg2 import pool

host = config["HOST"]
dbname = config["DBNAME"] 
user = config["USER"] 
password = config["PASSWORD"] 
sslmode = config["SSLMODE"] 

# Build a connection string from the variables
conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)

postgreSQL_pool = psycopg2.pool.SimpleConnectionPool(1, 20,conn_string)
if (postgreSQL_pool):
    print("Connection pool created successfully")

# Use getconn() to get a connection from the connection pool
connection = postgreSQL_pool.getconn()
cursor = connection.cursor()

In order to use pgvector, we need to first create the vector extension as described in this [link](https://learn.microsoft.com/en-us/azure/postgresql/flexible-server/how-to-use-pgvector) and shown below.

In [ ]:
# Open a cursor to perform database operations
# This is likely in case extension isn't already created from portal.
cursor = connection.cursor()

try:
    # Start a new transaction
    cursor.execute("BEGIN;")

    # Previous transaction statements
    # ...

    # Check if the extension already exists
    extension_query = "SELECT * FROM pg_extension WHERE extname = 'vector';"
    cursor.execute(extension_query)
    extension_exists = cursor.fetchone()

    if not extension_exists:
        # Extension does not exist, create it
        create_extension_query = "CREATE EXTENSION vector;"
        cursor.execute(create_extension_query)
        connection.commit()
    else:
        # Extension already exists, pass through
        pass

    # Commit the transaction
    cursor.execute("COMMIT;")
except Exception as e:
    # An error occurred, rollback the transaction
    cursor.execute("ROLLBACK;")
    raise e
finally:
    # Close the cursor
    cursor.close()


In the following we will list the existing extensions for your db. Please make sure ['VECTOR'] IS listed.

In [ ]:
import psycopg2

# Establish a connection to the database
connection = psycopg2.connect(conn_string)

# Open a cursor to perform database operations
cursor = connection.cursor()



# Define the SHOW EXTENSIONS query
show_extensions_query = "SHOW azure.extensions;"

# Execute the SHOW EXTENSIONS query
cursor.execute(show_extensions_query)

connection.commit()
# Fetch and print the results
results = cursor.fetchall()
for row in results:
    print(row)


[OPTIONAL] You may run the following to list the existing tables in your db 

In [ ]:
####### to get list of existing tables in the database
# Create a cursor object to interact with the database
cursor = connection.cursor()

# Execute the SQL query to retrieve the table names
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public'")

# Fetch all the results
table_names = cursor.fetchall()

# Print the table names
for table in table_names:
    print(table[0])
    
if not table_names:
    print("No table found")


If you wish to drop a db, please uncomment the following code and define the table_name you wish to remove from your db.

In [ ]:
# table_name = 'abc_v0'
# batch_size = 1000
# # Drop previous table of same name if one exists
# cursor.execute(f"DROP TABLE IF EXISTS {table_name};")
# print("Finished dropping table (if existed)")
# connection.commit()

### Load data into database
In this section, we will store two tables into db 1) {filter_id1_name}: maps filter_id values to DocId 2)ChunkEmbeddings. Let's first load the data into data frames.   

In [ ]:
import pandas as pd
df_chunks_embedding = pd.read_csv('../AnalyzedPDF/CombinedResults/ChunksEmbedding.csv')

In [ ]:
df_chunks_embedding.head(3)
#columns should look like the following with order preserved
#Id, Chunk, PageNumber, LineNumber, DocId, Embedding

Let's store embeddings dataframe into a table.

In [ ]:
from pgvector.psycopg2 import register_vector
from psycopg2 import Error
from psycopg2 import sql
import numpy as np
# Establish a connection to the database
connection = psycopg2.connect(conn_string)

# Register 'pgvector' type for the 'embedding' column
register_vector(connection)

# Convert the DataFrame to a list of tuples for bulk insertion
records = df_chunks_embedding.to_records(index=False)
records_list = records.tolist()

# Open a cursor to perform database operations
cursor = connection.cursor()

# Define the table name
table_name = 'EarningsCallChunksEmbedding'
batch_size = 100
# Drop previous table of same name if one exists
cursor.execute(f"DROP TABLE IF EXISTS {table_name};")
print("Finished dropping table (if existed)")
connection.commit()

# Execute the query to check if the table exists
cursor.execute(f"SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE table_name = '{table_name}');")

# Fetch the result
exists = cursor.fetchone()[0]

if exists:
    print(f"The table '{table_name}' exists in the database.")
    print("You may drop previous table (see commented code above) if you want to re-insert data.")
else:
    print(f"The table '{table_name}' does not exist in the database. Creating it now and inserting data ...")
    # Build a connection string from the variables
    conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)

    postgreSQL_pool = psycopg2.pool.SimpleConnectionPool(1, 20, conn_string)
    if postgreSQL_pool:
        print("Connection pool created successfully")

    # Use getconn() to get a connection from the connection pool
    with postgreSQL_pool.getconn() as connection:
        # Define the CREATE TABLE query
        create_table_query = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            Id INTEGER PRIMARY KEY,
            Ticker TEXT,
            Year INTEGER,
            Quarter INTEGER,
            Chunk TEXT,
            PageNumber INTEGER,
            LineNumber INTEGER,
            Embedding VECTOR
        );
        """
        # Execute the CREATE TABLE query
        cursor.execute(create_table_query)
        connection.commit()

        # Define the INSERT INTO query
        insert_query = f"INSERT INTO {table_name} (Id, Ticker, Year, Quarter, Chunk, PageNumber, LineNumber, Embedding) " \
                    f"VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"

        # Execute the INSERT INTO query for each row
        cursor.executemany(insert_query, records_list)
        connection.commit()
        
        # Execute the CREATE TABLE query
        try:
            with connection.cursor() as cursor:
                cursor.execute(create_table_query)
                connection.commit()
                print(f"Table {table_name} created successfully!")
        except (Exception, Error) as e:
            print(f"Error creating table {table_name}: {e}")
            connection.rollback()        
        
        
        # Convert numpy.int32 to int in each row
        records_list = [
            tuple(int(value) if isinstance(value, np.int32) else value for value in record)
            for record in records_list
        ]

        # Split the records list into batches
        batches = [records_list[i: i + batch_size] for i in range(0, len(records_list), batch_size)]

        # Iterate over each batch and perform bulk insert
        count = 0
        for batch in batches:
            count += 1
            print(f"Inserting batch {count} into the table")
            try:
                insert_query = sql.SQL(f"INSERT INTO {table_name} (Id, Ticker, Year, Quarter, Chunk, PageNumber, LineNumber, Embedding) " \
                                    f"VALUES ({', '.join(['%s'] * len(batch[0]))})")
                
                with connection.cursor() as cursor:
                    cursor.executemany(insert_query, batch)
                    connection.commit()
            except (Exception, Error) as e:
                print(f"Error inserting batch into the table: {e}")
                connection.rollback()


#### Example query

In [ ]:
# Assuming you have already established a connection and have a cursor object

# Rollback the current transaction
connection.rollback()
cursor = connection.cursor()
# Execute the SELECT statement

try:
    cursor.execute(f"SELECT count(Id) FROM {table_name};")
    rows = cursor.fetchall()
    for row in rows:
        print(f"Number of items: {row}")
except (Exception, Error) as e:
    print(f"Error executing SELECT statement: {e}")
    
try:
    cursor.execute(f"SELECT embedding FROM {table_name} limit 5;")
    rows = cursor.fetchall()
    for row in rows:
        print(f"Items ID: {row}")
except (Exception, Error) as e:
    print(f"Error executing SELECT statement: {e}")

try:
    cursor.execute(f"SELECT Ticker FROM {table_name} limit 1;")
    rows = cursor.fetchall()
    for row in rows:
        print(f"Items ID: {row}")
except (Exception, Error) as e:
    print(f"Error executing SELECT statement: {e}")

## User Asks a Question 
In this step, the code will convert the user's question to an embedding and then retieve the top K document chunks based on the users' question using the similarity. Please note that other similarity metrics can also be used.

In [ ]:
userQuestion = "What was the highest earning?"
filter_col = "Ticker"
filter_val = "MSFT"
retrieve_k = 3 # for retrieving the top k reviews from the database

In [ ]:
import openai
import pandas as pd
import pandas as pd
import numpy as np


openai.api_type = config["OPENAI_API_TYPE"] 
openai.api_key = config["OPENAI_API_KEY"]
openai.api_base = config["OPENAI_API_BASE"] 
openai.api_version = config["OPENAI_API_VERSION"] 


def createEmbeddings(text):
    response = openai.Embedding.create(input=text , engine=config["OPENAI_DEPLOYMENT_EMBEDDING"])
    embeddings = response['data'][0]['embedding']
    return embeddings

In [ ]:
# Generate embeddings for the question and retrieve the top k document chunks
questionEmbedding = createEmbeddings(userQuestion)

## Hybrid Search
In this case, we will first filter based on id range, and then do similarity search

In [ ]:
from pgvector.psycopg2 import register_vector

connection = psycopg2.connect(conn_string)
# Create a cursor after the connection
# Register 'pgvector' type for the 'embedding' column
register_vector(connection)
cursor = connection.cursor()
select_query = f"SELECT Id FROM {table_name}  WHERE {filter_col} = '{filter_val}' ORDER BY Embedding <-> %s LIMIT 3"
cursor = connection.cursor()
cursor.execute(select_query, (np.array(questionEmbedding),))
results = cursor.fetchall()

In [ ]:
# Use the top k ids to retrieve the actual text from the database 
top_ids = []
for i in range(len(results)):
    top_ids.append(int(results[i][0]))

print(top_ids)

Please note how the top ids are now different and within range.

#### Retrieve text from database

In [ ]:
# Assuming you have already established a connection and have a cursor object

# Rollback the current transaction
connection.rollback()

format_ids = ', '.join(['%s'] * len(top_ids))

sql = f"SELECT CONCAT('PageNumber: ', PageNumber, ' ', 'Text: ', Chunk) AS concat FROM {table_name} WHERE id IN ({format_ids})"

# Execute the SELECT statement
try:
    cursor.execute(sql, top_ids)    
    top_rows = cursor.fetchall()
    for row in top_rows:
        print(row)
except (Exception, Error) as e:
    print(f"Error executing SELECT statement: {e}")


# Offer Response to User's Question
In order to offer a response, a user can either follow a simple prompting method as shown below or leverage more sophisticated ways used by other libraries, such as [langchain](https://python.langchain.com/en/latest/index.html).

#### Prompting directly using Azure Open AI service

In [ ]:
# create a prompt template 
template = """
    context :{context}
    Answer the question based on the context above. Provide the page number associated with the answer as reference as well. If the
    information to answer the question is not present in the given context then reply "I don't know".
    Question: {query}
    Answer: """

In [ ]:
# create the context from the top_rows
context = ""
for row in top_rows:
    context += row[0]
    context += "\n"
    
print(context)

In [ ]:
print(userQuestion)
prompt = template.format(context=context, query=userQuestion)
print(prompt)

In [ ]:

response = openai.Completion.create(
    engine= config['OPENAI_DEPLOYMENT_COMPLETION'],
    prompt=prompt,
    max_tokens=1024,
    n=1,
    stop=None,
    temperature=1,
)

print("prompt: ", prompt)
print('~~~~~')
# print("response: ", response['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip())
print(response['choices'][0]['text'])

